In [38]:
from random import randrange
from csv import reader
from math import sqrt

In [39]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [40]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [41]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [76]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [77]:
cross_validation_split(load_csv('db_hsv_all.csv'),10)

[[['1.76', '0.5', 'teko_heater'],
  ['1.795', '0.49', 'penghapus'],
  ['1.785', '0.5', 'jam_weker']],
 [['1.775', '0.5', 'panci'],
  ['1.79', '0.5', 'penghapus'],
  ['1.75', '0.5', 'jangka_sorong']],
 [['1.77', '0.49', 'jangka_sorong'],
  ['1.78', '0.5', 'jam_weker'],
  ['1.785', '0.5', 'oli']],
 [['166.5', '0.5', 'gelas_plastik'],
  ['1.76', '0.5', 'jangka_sorong'],
  ['1.785', '0.5', 'jam_weker']],
 [['1.785', '0.5', 'oli'],
  ['1.785', '99.5', 'botol_putih'],
  ['1.795', '0.5', 'teko_heater']],
 [['241.425', '99.5', 'tes_celup'],
  ['1.775', '0.5', 'panci'],
  ['1.76', '0.5', 'botol_putih']],
 [['246.725', '0.5', 'tes_celup'],
  ['1.78', '0.5', 'panci'],
  ['260.475', '99.5', 'liquid']],
 [['1.7', '0.5', 'penghapus'],
  ['1.755', '0.5', 'botol_putih'],
  ['1.65', '0.5', 'liquid']],
 [['166.5', '0.47', 'gelas_plastik'],
  ['1.785', '0.5', 'oli'],
  ['1.755', '0.5', 'tes_celup']],
 [['1.78', '0.5', 'teko_heater'],
  ['266.71', '99.5', 'liquid'],
  ['1.5', '0.47', 'gelas_plastik']]]

In [78]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [79]:
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

In [80]:
# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
	predictions = list()
	for row in test:
		output = predict_classification(train, row, num_neighbors)
		predictions.append(output)
	return(predictions)

In [82]:
# Test the kNN on dataset
filename = 'db_hsv_all.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 10
num_neighbors = 1
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [33.33333333333333, 33.33333333333333, 0.0, 33.33333333333333, 33.33333333333333, 33.33333333333333, 66.66666666666666, 0.0, 0.0, 33.33333333333333]
Mean Accuracy: 26.667%
